In [1]:
# Import required Packages for the chapter
import numpy as np

# 3 Solving linear systems of equations
<br>
<ul>
    <li> Linear systems of equations arise in almost every branch of engineering and science.</li>
    <li> We will eventually see that the need for numerical methods for solving linear systems arise in solving boundary value differential equations and as part of nonlinear equation solvers.</li>
    <li> In this chapter we will be learning about the Gaussian elimination method which along with its variants are best tecniques in practice for systems that involve small dense matrices.</li>
    For large sparse linear systems, iterative solvers like Conjugate Gradient and GMREs are typically better choices.
</ul>
<br>
<b><font size="+1"> 3.1 Linear systems of equations and solvability</font></b><br>
A linear system of three variables given as <br><br>
$\hspace{10mm} \begin{align*}a_{11}x_1+a_{12}x_2+a_{13}x_3 &=b_1\\ a_{21}x_1+a_{22}x_2+a_{23}x_3 &=b_2\\ a_{31}x_1+a_{32}x_2+a_{33}x_3 &=b_3\end{align*}$<br><br>
can be be written in the matrix form as<br><br>
$\hspace{10mm} \begin{pmatrix} a_{11} & a_{12} & a_{13} \\ a_{21} & a_{22} & a_{23} \\ a_{31} & a_{32} & a_{33} \end {pmatrix} \hspace{2mm} \begin{pmatrix}x_1\\ x_2\\ x_3 \end{pmatrix} = \begin{pmatrix}b_1\\ b_2\\ b_3 \end{pmatrix} \hspace{10mm} \Longleftrightarrow \hspace{10mm} \large{Ax = b}$<br><br>
where $A$ is a $3 \times 3$-matrix with coefficients from the left-hand side of the system as entries, $x$ is $3 \times 1$-vector with the unknowns as entries and $b$ is a $3 \times 1$-vector with the constants from the right-hand side of the system as entries.<br><br><br>
Any system of linear equations in $\large{n}$ variables, $\large{A_{_{n \times n}}} x_{_{n \times 1}} = b_{_{n \times 1}}$ can have three posibilities (explained using examples in two variables but the ideas extend to any number of variables):
<ul>
    <li><b>No solutions</b>: When equations are inconsistent, i.e., one or more equations disagree with each other or rest of the system.<br>
    Example: $\begin{align*} 2x_1+x_2 &=0\\ 2x_1+x_2 &=1 \end{align*}$<br>
    Geometrically this means that the two straight lines represented by the equations in $xy$-plane represent two parallel straight lines and hence never intersect.
    </li>
    <li><b>Infinite solutions</b>: When one or more of the equations are redundant, i.e., one or more of the equations can be written as a combination of the other equations.<br>
    Example: $\begin{align*} 2x_1+x_2 &=1\\ 4x_1+2x_2 &=2 \end{align*}$<br>
    Geometrically, this means that both equations represent same staright line and every point on the straight line is a solution to the system.
    </li>    
    <li><b>Unique solution</b>: The equations are consistent and all the $n$ equations are essential.<br>
    Example: $\begin{align*} 2x_1+x_2 &=3\\ x_1-2x_2 &= 4\end{align*}$<br>
    Geometrically, this means that both equations represent two unique non-parallel staright lines that intersect at an unique point.
    </li>    
</ul>

<b><font size="+1"> 3.2 Solving Triangular Systems</font></b><br>
First step to solving general linear systems of equations is to solve 'triangular systems'. As the name suggests, in the matrix representation of the coefficients of the system, the matrix $A$ on the LHS has all $0$'s either above or below its main diagonal. Finding solution to such a system is easy. The process is known as <b> Back-substitution.</b> <br><br>
<b>Example : </b>Back-substitution for an upper triangular system in three variables is explained using the following example.<br><br>
Let us solve a upper triangular system for a system with three variavles using back-substitution.<br><br>
$\hspace{10mm} \begin{pmatrix} 1 & 2 & 3 \\ 0 & 4 & 5 \\ 0 & 0 & 6 \end {pmatrix} \hspace{2mm} \begin{pmatrix}x_1\\ x_2\\ x_3 \end{pmatrix} = \begin{pmatrix}1\\ -5\\ -6 \end{pmatrix} \Longleftrightarrow \begin{align*}x_1+2x_2+3x_3 &=1\\ 4x_2+5x_3 &=-5\\ 6x_3 &=-6\end{align*}$<br><br>
The last equation solves for $x_3=-1$ which reduces the first two equations to the new system in two variables,<br><br>
$\hspace{10mm}\begin{pmatrix} 1 &2 \\ 0 &4 \\ \end{pmatrix} \begin{pmatrix} x_1\\ x_2 \end{pmatrix} = \begin{pmatrix} 1-3x_3 \\ -5-5x_3\end{pmatrix} = \begin{pmatrix} 4\\ 0 \end{pmatrix}$<br><br>
Once again repeating the process we can readily obtain the value of $x_2=0$ from the last equation of the newly obtained system. If we replace this value back into the system, we obtain an equation in just $x_1$ and hence the value $x_1=4$. $\square$

In [2]:
# Back-substitution to solve triangular system
def BackSubstitution(A,b):
    n=len(b)
    x=np.zeros(n)

    for i in range(n-1,-1,-1):
        # Check if the matrix is singular
        if A[i][i]==0:
            print("Matrix Singularity: Triangular Matrix has zero in diagonal")
            return

        # Compute solution
        x[i] = b[i]/A[i][i]

        # Update the RHS vector
        for j in range(i):
            b[j]=b[j]-A[j][i]*x[i]

    return x


In [3]:
A=np.array([[1,2,3],[0,4,5],[0,0,6]])
b=np.transpose(np.array([1,-5,-6]))
print("Solution : ",BackSubstitution(A,b))

Solution :  [ 4.  0. -1.]


<b><font size = "+1">3.3 Gaussian elimination</font></b><br>
The Gaussian elimination is a two-step process that consists of:
<ul>
    <li> Transforming the given system to an upper tringular system.</li>
    <li> Use back-substitution to solve this upper triangular system.</li>
</ul>
<br>
To convert the system to an upper triangular system, we first transform the matrix $A$ to an upper triangular matrix $U$ through a series of elementary row operations. The row operations used are in the form $R_j = R_j + kR_i$, that is $j$-th row of the matrix is replaced by the sum of $j$-th row and $k$ times the $i$-th row, where $k$ is a constant. The multiplyers "$k$" is so chosen that any element below the diagonal position $(i,i)$ to become zero.<br><br>
Note that the above transformation corresponding to the elementary row operation and equivalent to left-multiplying matrix $A$ by a matrix $E$ of the same order $n \times n$, which has $k$ as its entry at position $(j,i)$, $1$s for its diagonal entries and $0$s for all other entries. In other words, this matrix $E$ is obtained by replacing the $zero$ at the position $(j,i)$ of an identity matrix by $k$. This matrix $E$ is called an elementary matrix since it corresponds to an elementary row operation.

We will explain the Gaussian elimination using the following example.<br><br>
<b>Example</b> : Consider the matrix $A= \begin{pmatrix} 2&1&-3\\4&1&2\\-5&3&7\\ \end{pmatrix}$
<br><br>
In the first step we choose $k = -\frac{a_{21}}{a_{11}}$. This leads to the first element of $R_2$ to become zero. Lets call the new matrix we obtain as $A^{(1)}$, whose elements we shall denote by $a^{(1)}_{ij}$. See how the other elements in this row are affected but they may not necessarily become zero. <br><br>
$A=\begin{pmatrix} 2&1&-3\\4&1&2\\-5&3&7\\ \end{pmatrix}  \xrightarrow[]{\large{R_2=R_2+(-2)R_1}} \underbrace{\begin{pmatrix} 2&1&-3\\ 0&-1&8\\ -5&3&7 \end{pmatrix}}_{A^{(1)}}$<br><br>
In the second step we choose $k = -\frac{a^{(1)}_{31}}{a^{(1)}_{11}}$. This leads to the first element of $R_3$ to become zero. Call the new matrix $A^{(2)}$. At this point we went through the first column making all the entries below position $(1,1)$ to be zero.<br><br>
$\hspace{30mm} A^{(1)}\xrightarrow[]{\large{R_3=R_3+(2.5)R_1}} \underbrace{\begin{pmatrix} 2&1&-3\\ 0&-1&8\\ 0&5.5&-0.5 \end{pmatrix}}_{A^{(2)}}$<br>
We want the elements below $a^{(2)}_{22}$ to be zero. Hence, in this third and final step we choose $k = -\frac{a^{(2)}_{32}}{a^{(2)}_{22}}$. This leads to the first non-zero element of $R_3$ at position $(3,2)$ to become zero. Note that none of the zeros in the first column are affected by this operation.<br><br>
$\hspace{30mm} A^{(2)}\xrightarrow[]{\large{R_3=R_3+(5.5)R_2}} \underbrace{\begin{pmatrix} 2&1&-3\\ 0&-1&8\\ 0&0&43.5 \end{pmatrix}}_{\text{ upper triangular matrix }U}$<br><br>
With this step we have obtained the desired upper triangular matrix. $\square$

From the above example we can see that at any step starting with a matrix $A^{(r)}$, if we apply the row operation $R_j = R_j + kR_i$ then we choose $\large k = -\left(\frac{a^{(r)}_{ji}}{a^{(r)}_{ii}}\right)$.<br>
If we apply these row operations in the same order to both $A$ and $b$ we obtain a new upper triangular system whose solution is same as the solution to the original linear system. As mentioned before, the elementary row operation is equivalent to left-multiplication by an elementary matrix. Hence the series of row operation is equivalent to being multiplied by a matrix $P=E_n...E_2E_1$<br>
i.e., the original linear system is rewritten as the following upper triangular system,
$$\large Ax=b \Leftrightarrow P(Ax)=Pb \Leftrightarrow Ux=b', \hspace{10mm} \small \text{where }U=PA \text{ and }b'=Pb $$
This new system has the same solution as the original system.
Hence, at this point we use the back-substitution to solve this new system and obtain the required solution.

<b>Example</b> : Use Gaussian elimination to solve to system <br><br>
$\hspace{50mm} \begin{align*} x+y+z&=2\\ 6x−4y+5z&=31\\ 5x+2y+2z&=13 \end{align*}$<br><br>
Step 1: Transform the given system to upper triangular system. In this case we will be applying the row transformation to both matrix A and b. Convention is to write the matrix and vector in the following form known as the <b>augmented matrix</b> to apply the row transformations.<br><br>
$\hspace{50mm}
\begin{align*}
[A|b]=\left(\begin{array}{ccc|c} 1 & 1 & 1 & 2 \\ 6 & -4 & 5 & 31\\ 5 & 2 & 2 & 13 \end{array}\right)
&\xrightarrow[]{\large{R_3=R_3+(-6)R_1}}
\left(\begin{array}{ccc|c} 1 & 1 & 1 & 2 \\ 0 & -10 & -1 & 19\\ 5 & 2 & 2 & 13 \end{array}\right)\\
&\xrightarrow[]{\large{R_3=R_3+(-5)R_1}}
\left(\begin{array}{ccc|c} 1 & 1 & 1 & 2 \\ 0 & -10 & -1 & 19\\ 0 & -3 & -3 & 3 \end{array}\right)\\
&\xrightarrow[]{\large{R_3=R_3+(-0.3)R_1}}
\left(\begin{array}{ccc|c} 1 & 1 & 1 & 2 \\ 0 & -10 & -1 & 19\\ 0 & 0 & -2.7 & -2.7 \end{array}\right)
\end{align*}
$
<br><br>
Step 2: Use back-substitution to get the solution.<br><br>
$\hspace{50mm}
\begin{align*}
z &=\frac{-2.7}{-2.7}=1\\
y &=\frac{19 - (-1)z}{-10} =-2 \\
x &=\frac{2 - (1)x -(1)z}{1} =3 \\
\end{align*}$

<font size = "+1"><b>Pivoting</b></font><br>
The process of Gaussian elimination requires a row operation $R_j = R_j + kR_i$ where $\large k = - \left(^{a^{(r)}_{ji}}\big/_{a^{(r)}_{ii}}\right)$.That is, the step requires divison by the diagonal element. If the diagonal element turns into a zero on account of a row transformation, the process will fail due to divison by zero. To avoid this situation we emply the process of pivoting. The process involves a new kind of elementary row operation $R_i \leftrightarrow R_j$ which interchanges the $i$-th row of a matrix with the $j$-th row.<br>
Note that the above row operation is equivalent to left-multiplication by an elementary matrix $E$ which is obtained by interchanging the $i$-th and $j$-th rows of an identity matrix.<br><br>
The way pivoting works, is that, when we obtain a zero for the diagonal element during the Gaussian elimination process, say at $i$-th row, we scan this $i$-th column of the matrix to find the first non-zero element below that diagonal element. Say we find this element at row $j$. This is when we swap the rows $i$ and $j$ to get a non-zero element at the diagonal position $(i,i)$. Failure of this pivoting process means that we were unable to find a non-zero element below the diagonal, and since the diagonal element is also zero, this means the system is either inconsistent or has infinite solutions. In either case, the Gaussian elimination process fails to produce solutions since unique solutions do not exist.  

The following example explains the process of Gaussian elimination with pivoting.<br><br>
<b>Example </b>: Consider the following system of linear equations,<br><br>
$ \hspace{50mm} \begin{align*} x+y+2z &= 9\\ 2x+2y-3z &=-3\\ -5x+y+4z &=9 \end{align*}$<br><br>
We apply the Gaussian elimination on the augmented matrix.<br><br>
$\begin{align*}
[A|b] = \left(\begin{array}{ccc|c} 1 & 1 & 2 & 9\\ 2 & 2 & -3 & -3\\ -5 & 1 & 4 & 9\end{array}\right)
& \xrightarrow[]{R_2 = R_2 + (-2)R_1}
\left(\begin{array}{ccc|c} 1 & 1 & 2 & 9\\ 0 & 0 & -7 & -21\\ -5 & 1 & 4 & 9 \end{array}\right)\\
& \xrightarrow[]{R_2 = R_2 + (5)R_1}
\left(\begin{array}{ccc|c} 1 & 1 & 2 & 9\\ 0 & 0 & -7 & -21\\ 0 & 6 & 14 & 54\end{array}\right)\\
\end{align*}$<br><br>
At this point we notice that there is a zero at position $(2,2)$ hence we cannot process until we employ pivoting. If we traverse down this column, we find the fist non-zero element below the diagonal at position $(3,2)$. So we interchange the row-$2$ with row-$3$ before continuing with the Gaussian elimination.<br><br>
$\hspace{56mm}
\begin{align*}
&\xrightarrow[]{R_2 \leftrightarrow R_3}
\left(\begin{array}{ccc|c} 1 & 1 & 2 & 9\\ 0 & 6 & 14 & 54 \\ 0 & 0 & -7 & -21\end{array}\right)\\
\end{align*}$<br><br>
SInce at this point we have a upper triangular system, we can apply the back substitution to obtain the solution as $$ x=1, y=2, z=3. \square$$

<b>Exercise </b>: Use Gaussian elimination with pivoting to solve the following linear system of equations,<br><br>
$\hspace{50mm} \begin{align*} 2y-3z &= -5\\ 2x+y+4z &=16\\ 2x+y-z &=1 \end{align*}$

In [4]:
def firstIndexForNonZero(A):
    n=len(A[:,0])
    for i in range(n):
        if A[i][0]>=10**(-10):
            return i
        if i==n-1:
            return -1

def pivot(A,b):
    if firstIndexForNonZero(A) == -1:
        print('Matrix Singularity: Pivoting not possible')
        return 0  # this indicates that pivoting has failed
    else:
        i=firstIndexForNonZero(A)

        A[0]=A[0]+A[i]
        A[i]=A[0]-A[i]
        A[0]=A[0]-A[i]

        b[0]=b[0]+b[i]
        b[i]=b[0]-b[i]
        b[0]=b[0]-b[i]

    return 1    # this indicates that pivoting has been successful

In [5]:
b=np.array([1,3,5,4])
A=np.array([[0,0,3,1],[0,5,1,6],[2,8,9,1],[3,7,2,1]])
firstIndexForNonZero(A)
pivot(A,b)
A,b

(array([[2, 8, 9, 1],
        [0, 5, 1, 6],
        [0, 0, 3, 1],
        [3, 7, 2, 1]]),
 array([5, 3, 1, 4]))

In [6]:
A[1:4,0:3],b[1:4]

(array([[0, 5, 1],
        [0, 0, 3],
        [3, 7, 2]]),
 array([3, 1, 4]))

In [7]:
pivot(A[1:4,0:3],b[1:4])

1

<b> Gaussian Elimination </b><br>
Gaussian elimination is the process of transforming a given linear system to a upper triangular system and then solve the system using back-susbtitution.

In [19]:
def GaussianElimination(A,b):
    n=len(b)
    print('Given System : \n')
    for i in range(n):
        print(A[i]," | ",b[i])

    for i in range(n-1):
        if A[i][i] == 0:
            piv = pivot(A[i:n][i:n],b[i:n])
            if piv == 0:
              print('Matrix Singularity: Gaussian Elimination not possible')
              return
        for j in range(i+1,n):
            m=A[j][i]/A[i][i]
            A[j,:]=A[j,:]-m*A[i,:]
            b[j]=b[j]-m*b[i]
    print('\nNew Upper Triangular System : \n')
    for i in range(n):
        print(A[i]," | ",b[i])
    print("\n")
    return BackSubstitution(A,b)

In [20]:
b=np.array([9,16,6])
A=np.array([[0,1,4],[2,4,6],[5,6,0]])
print("Solution : ",GaussianElimination(A,b))

Given System : 

[0 1 4]  |  9
[2 4 6]  |  16
[5 6 0]  |  6

New Upper Triangular System : 

[2 4 6]  |  16
[0 1 4]  |  9
[0 0 1]  |  2


Solution :  [0. 1. 2.]


In [24]:
b=np.array([10.0,17.0])
A=np.array([[6,-2],[11.5,-3.85]])
print("Solution : ",GaussianElimination(A,b))

Given System : 

[ 6. -2.]  |  10.0
[11.5  -3.85]  |  17.0

New Upper Triangular System : 

[ 6. -2.]  |  10.0
[ 0.         -0.01666667]  |  -2.166666666666668


Solution :  [ 45. 130.]


In [25]:
b=np.array([10.0,17.0])
A=np.array([[6,-2],[11.5,-3.84]])
print("Solution : ",GaussianElimination(A,b))

Given System : 

[ 6. -2.]  |  10.0
[11.5  -3.84]  |  17.0

New Upper Triangular System : 

[ 6. -2.]  |  10.0
[ 0.         -0.00666667]  |  -2.166666666666668


Solution :  [110. 325.]


In [10]:
b=np.array([9,16,6])
A=np.array([[0,1,4],[0,4,6],[0,6,0]])
print("Solution : ",GaussianElimination(A,b))

Given System : 

[0 1 4]  |  9
[0 4 6]  |  16
[0 6 0]  |  6
Matrix Singularity: Pivoting not possible
Matrix Singularity: Gaussian Elimination not possible
Solution :  None


<b>Complexity</b><br>
We estimate the work done by the algorithm by counting the arithmetic operations in each loop. On the $k$-th time through the outer loop, we perform:<br>


<b><font size ="+1">LU Decomposition</font></b><br><br>
Most matrices $A$ can be decomposed into a product $A=LU$ where $L$ is a lower triangular matrix and $U$is an upper triangular matrix. (<i>when?</i>) <br>
This is known as LU decomposition oof $A$. These matrices $L$ and $U$ can be found as a result of Gaussian elimination method (provided no pivoting is done, otherwise, it wil be $A=PLU$).(<i>how?</i>)<br>
The following equation shows how the LU decompositon of a $3\times 3$ matrix $A$ looks like.
<br><br>
$\begin{align*}
\begin{pmatrix}a_{11}&a_{12}&a_{13}\\ a_{21}& a_{22}&a_{23}\\a_{31}&a_{32}&a_{33}\end{pmatrix} &=\begin{pmatrix}1&0&0\\ l_{21}& 1&0 \\ l_{31}&l_{32}&1\end{pmatrix}\begin{pmatrix}u_{11}& u_{12}&u_{13}\\ 0 & u_{22}&u_{23}\\0&0&u_{33}\end{pmatrix} \\
&= \begin{pmatrix} u_{11}& u_{12}&u_{13}\\ l_{21}u_{11} & l_{21}u_{12}+u_{22} & l_{21}u_{13}+u_{23} \\ l_{31}u_{11} & l_{31}u_{12}+l_{32}u_{22} & l_{31}u_{13}+l_{32}u_{23}+u_{33} \end{pmatrix}
\end{align*}$<br><br>

One method of obtaining the $L$ and $U$ matrices is to solve for the values of the entries. Note that the diagonal entries of the matrix $L$ has been deliberately chosen as $1$s to obtain a unique solution. The solution turns out to be the following:<br><br>
$\begin{align*}
& --- &  & u_{11} =a_{11} &      & u_{12} =a_{12} &       & u_{13} =a_{13} \\
& l_{11} =\frac{a_{11}}{u_{11}} &    & --- &         & u_{22} =a_{22}-l_{21}u_{12} &    & u_{23} =a_{23}-l_{21}u_{13} \\
& l_{21} =\frac{a_{21}}{u_{11}} &    & l_{22} = \frac{a_{22}-l_{21}u_{12}}{u_{22}} &        & ---  &     & u_{33} =a_{33}-l_{31}u_{13}-l_{32}u_{23} \\
& l_{31} =\frac{a_{31}}{u_{11}} &    & l_{32} =\frac{a_{32}-l_{31}u_{12}}{u_{22}} &       & l_{33}  =\frac{a_{33}-l_{31}u_{13}-l_{32}u_{23}}{u_33} &  &---\\
\end{align*}$
<br><br>

The following algorithm gives us one possible method to compute the $L$ and $U$ matrices using the above solutions.<br><br>
$\hspace{10mm} \text{for}\; 0 \leq i \leq n-1:\\
\hspace{20mm} \text{for}\; 0 \leq j \leq n-1:\\
\hspace{30mm} \text{if}\; i==0\; \text{then}\\
\hspace{40mm} u_{ij}=a_{ij}\\
\hspace{40mm} \text{if } i==j \text{ then } L[i][i]=1 \; \text{ else } l_{ji}=\frac{a_{ji}}{u_{ii}}\\
\hspace{30mm} \text{if }\; i > 0\; \text{ then }\\
\hspace{40mm} sum_1=\sum\limits_{k=0}^{i-1} l_{ik}u_{kj}; \; u_{ij}=a_{ij}-sum_1\\
\hspace{40mm} \text{if } i==j \text{ then } L[i][i]=1 \; \text{ else } sum_2=\sum\limits_{k=0}^{i-1} l_{jk}u_{ki}; \; l_{ji}=\frac{a_{ji}-sum_2}{u_{ii}}\\
\hspace{20mm} \text{end for}\\
\hspace{10mm} \text{end for}
$

In [11]:
def LUDecomp(A):
    n=len(A)
    L=np.zeros([n,n])
    U=np.zeros([n,n])
    for i in range(n):
        for j in range(i,n):
            if i == 0 :
                U[i][j]=A[i][j]
                if i==j:
                    L[i][i]=1
                else:
                    L[j][i]=A[j][i]/U[i][i]
            if i>0 :
                sum1=0
                for k in range(i):
                    sum1 += L[i][k]*U[k][j]
                U[i][j] = A[i][j] - sum1
                if i==j:
                    L[i][i]=1
                else:
                    sum2=0
                    for k in range(i):
                        sum2 += L[j][k]*U[k][i]
                        L[j][i] = (A[j][i]-sum2)/U[i][i]
    return L,U

In [12]:
A=np.array([[1,1,2],[1,5,4],[-2,-3,11]])
L,U =LUDecomp(A)
print("A = \n",A)
print("L = \n",L)
print("U = \n",U)

A = 
 [[ 1  1  2]
 [ 1  5  4]
 [-2 -3 11]]
L = 
 [[ 1.    0.    0.  ]
 [ 1.    1.    0.  ]
 [-2.   -0.25  1.  ]]
U = 
 [[ 1.   1.   2. ]
 [ 0.   4.   2. ]
 [ 0.   0.  15.5]]


Note that the method works well as long as any divison by zero does not appear while calculating the entries for $L$. Which means none of the diagonal entries of $U$, i.e. $u_{ii}$s should be zero. In which case the above method fails.

In [13]:
A=np.array([[0,1,2],[0,5,4],[0,-3,11]])
L,U =LUDecomp(A)
print("A = \n",A)
print("L = \n",L)
print("U = \n",U)

A = 
 [[ 0  1  2]
 [ 0  5  4]
 [ 0 -3 11]]
L = 
 [[ 1.  0.  0.]
 [nan  1.  0.]
 [nan nan  1.]]
U = 
 [[ 0.  1.  2.]
 [ 0. nan nan]
 [ 0.  0. nan]]


<ipython-input-11-2145adc8a51f>:12: RuntimeWarning: invalid value encountered in double_scalars
  L[j][i]=A[j][i]/U[i][i]


Notice the error message along with '$nan$'s in the resultant matrices above.<br>
A more robust technique for finding the LU-Decomposition for a matrix is to use the Gaussian elimination method.

<b>Example</b>: Create an LU decompostion of $A=\begin{pmatrix} 1&1&2\\ 1&5&4\\ -2&-3&11 \end{pmatrix}$. <br><br>
We start by applyting Gaussian elimination method on $A$ to obtain the upper triangular matrix $U$. We apply the following transformations:
<br><br>
$\begin{align*}
A=\begin{pmatrix} 1&1&2\\ 1&5&4\\ -2&-3&11 \end{pmatrix} & \xrightarrow[]{\large{R_2=R_2+(-1)R_1}} &\begin{pmatrix} 1&1&2\\ 0&4&2\\ -2&-3&11 \end{pmatrix}\\
& \xrightarrow[]{\large{R_3=R_3+(2)R_1}} &\begin{pmatrix} 1&1&2\\ 0&4&2\\ 0&-1&15 \end{pmatrix}\\
& \xrightarrow[]{\large{R_3=R_3+\big(\frac{1}{4}\big)R_2}} &\begin{pmatrix} 1&1&2\\ 0&4&2\\ 0&0&15.5 \end{pmatrix}\\
\end{align*}$
<br><br>
To create the lower triangular matrix, we start with the identity matrix $I_{3 \times 3}$ and apply the column operations that would create the reverse process of the row operations used in the Gaussian elimination.<br>
For example, note that $R_3=R_3+(2)R_1$ is equivalent to left multiplication by the matrix $\begin{pmatrix} 1&0&0\\ 0&1&0\\ 2&0&1 \end{pmatrix}$ and the inverse of the matrix is $\begin{pmatrix} 1&0&0\\ 0&1&0\\ -2&0&1 \end{pmatrix}$ which we right-multiply to the identity matrix. This multiplication is equivalent to the column operation $C_1=C_1+(-2)C_3$ on the identity matrix. Notice that the negative of the multiplyer from the row operation is used in the column operation and the indices have switched compared to the row operations. <br><br>
Hence the transformations we apply in the following order to the identity matrix are <br>
$\begin{align*} C_1 &=C_1+(1)C_2, \\ C_1 &=C_1+(-2)C_3 \hspace{5mm} \text{  and}\\ C_2 &=C_2+\big(-\frac{1}{4} \big)C_3 \end{align*}$ <br>
which transforms it to $L=\begin{pmatrix} 1&0&0\\ 1&1&0\\ -2&{-\frac{1}{4}}&1 \end{pmatrix}$.<br>
Notice how the negative values of the multiplyers from the row operation in the Gaussian elimination replaces the zeros in the triangle below the diagonal of the identity matrix. One can easily verify that $A=LU$. $\square$
<br><br>        
$\begin{align*}
\text{Note that, } \hspace{5mm}A &=IA\\
&= (IE_{1}^{-1})(E_{1}A) \hspace{5mm}\text{[using elementary operations] }  \\
& \cdots\\
&= (IE_{1}^{-1}E_{2}^{-1}\cdots E_{n}^{-1})(E_{n}\cdots E_{2}E_{1}A)\\
&= I(E_{n}\cdots E_{2}E_{1})^{-1}(E_{n}\cdots E_{2}E_{1})A\\
&= (IP^{-1})(PA) \\
&=LU\\
\end{align*}$<br>
Hence we if we apply a bunch of elementary row operations each of the form $R_j = R_j + k R_i$ to apply the process of Gaussian elimination, we can obtain to upper triangular matrix $U=PA$.<br>
To obtain the lower triangular matrix $L=IP^{-1}$ we have to apply the equivalent column operations on the identity matrix to get the inverse, which turns out to be $C_i = C_i - k C_j$ for each corresponding elementary row operation.<br>
This just means replacing the $zero$ by the factor "$-k$" at position $(j,i)$ of the lower triangular matrix obtained so far by transformaton of the identity matrix.

In [14]:
def LUDecompose(A):
    n=len(A)
    L=np.eye(n).astype(float)
    U=np.array([row[:] for row in A]).astype(float)
    print(U)
    for i in range(n-1):
        if U[i][i]==0:
            continue
        else:
            for j in range(i+1,n):
                factor=-(U[j][i]/U[i][i])
                U[j,:]=U[j,:] + (factor)*U[i,:] # row operation for gaussian elimination
                L[j][i]=-(factor) # column operation equivalent to the inverse of the elementary row operation
                                                # used in the above step
    return L,U

In [15]:
A=np.array([[1,1,2],[1,5,4],[-2,-3,11]])
L,U =LUDecompose(A)
print("A = \n",A)
print("L = \n",L)
print("U = \n",U)

[[ 1.  1.  2.]
 [ 1.  5.  4.]
 [-2. -3. 11.]]
A = 
 [[ 1  1  2]
 [ 1  5  4]
 [-2 -3 11]]
L = 
 [[ 1.    0.    0.  ]
 [ 1.    1.    0.  ]
 [-2.   -0.25  1.  ]]
U = 
 [[ 1.   1.   2. ]
 [ 0.   4.   2. ]
 [ 0.   0.  15.5]]


In [16]:
A=np.array([[0,1,2],[0,5,4],[0,-3,11]])
L,U =LUDecompose(A)
print("A = \n",A)
print("L = \n",L)
print("U = \n",U)

[[ 0.  1.  2.]
 [ 0.  5.  4.]
 [ 0. -3. 11.]]
A = 
 [[ 0  1  2]
 [ 0  5  4]
 [ 0 -3 11]]
L = 
 [[ 1.   0.   0. ]
 [ 0.   1.   0. ]
 [ 0.  -0.6  1. ]]
U = 
 [[ 0.   1.   2. ]
 [ 0.   5.   4. ]
 [ 0.   0.  13.4]]
